In [55]:
import pandas as pd

In [56]:
df = pd.read_csv("data.csv")

df.head()

,id,device,created_at,updated_at,guid,externalhumidity,externaltemp,internaltemp,lightintensity,windspeed,winddirection,rainfall,pressure,soilmoisture,altitude,battery
0,75371,12,2026-02-18T18:13:36.698Z,2026-02-18T18:13:36.698Z,2851e9a0-b608-46,77.38,24.81,29.0,0.0,1.32,19.88,0,100734.07,0,49.41,9.30
1,75370,12,2026-02-18T18:11:14.434Z,2026-02-18T18:11:14.434Z,2851e9a0-b608-46,77.06,24.84,29.0,0.0,2.53,7.07,0,100734.20,0,49.00,9.50
2,75369,12,2026-02-18T18:10:43.763Z,2026-02-18T18:10:43.763Z,2851e9a0-b608-46,77.08,24.81,29.0,0.0,3.09,93.64,0,100736.63,0,49.15,9.55
3,75368,12,2026-02-18T18:10:13.036Z,2026-02-18T18:10:13.036Z,2851e9a0-b608-46,77.12,24.81,29.0,0.0,1.97,95.40,0,100734.34,0,49.35,9.48
4,75367,12,2026-02-18T18:09:42.399Z,2026-02-18T18:09:42.399Z,2851e9a0-b608-46,77.16,24.88,29.0,0.0,1.84,95.40,0,100733.27,0,49.23,9.59


In [57]:
df.isnull().sum()
print(df.describe())


                 id   device  externalhumidity  externaltemp  internaltemp  \
count  70551.000000  70551.0      70551.000000  70551.000000  70551.000000   
mean   40096.000000     12.0         69.619382     27.413780     33.514982   
std    20366.463758      0.0         10.417365      2.256673      4.005555   
min     4821.000000     12.0          0.000000      0.000000      0.000000   
25%    22458.500000     12.0         62.820000     25.720000     30.000000   
50%    40096.000000     12.0         69.760000     27.380000     33.000000   
75%    57733.500000     12.0         76.715000     29.190000     37.000000   
max    75371.000000     12.0         95.030000     34.940000     44.500000   

       lightintensity     windspeed  winddirection  rainfall       pressure  \
count    70551.000000  70551.000000   70551.000000   70551.0   70551.000000   
mean     18963.706757      3.201138      51.815328       0.0  100762.067678   
std      21675.337589      1.690073      48.537387       0.0

In [58]:
cols_to_drop = ['id', 'guid', 'updated_at', 'rainfall', 'soilmoisture']
df.drop(columns=cols_to_drop, inplace=True)

print("Remaining columns:", df.columns.tolist())

Remaining columns: ['device', 'created_at', 'externalhumidity', 'externaltemp', 'internaltemp', 'lightintensity', 'windspeed', 'winddirection', 'pressure', 'altitude', 'battery']


In [59]:
df['created_at'] = pd.to_datetime(df['created_at'], utc=True)
df.sort_values('created_at', inplace=True)
df.set_index('created_at', inplace=True)

In [60]:
df.head


<bound method NDFrame.head of                                   device  externalhumidity  externaltemp  \
created_at                                                                 
2025-12-21 12:00:05.028000+00:00      12          68.33000         27.94   
2025-12-21 12:00:35.882000+00:00      12          68.20000         27.94   
2025-12-21 12:05:16.027000+00:00      12          68.43999         27.91   
2025-12-21 12:05:36.205000+00:00      12          68.38000         27.88   
2025-12-21 12:06:06.926000+00:00      12          68.24000         27.94   
...                                  ...               ...           ...   
2026-02-18 18:09:42.399000+00:00      12          77.16000         24.88   
2026-02-18 18:10:13.036000+00:00      12          77.12000         24.81   
2026-02-18 18:10:43.763000+00:00      12          77.08000         24.81   
2026-02-18 18:11:14.434000+00:00      12          77.06000         24.84   
2026-02-18 18:13:36.698000+00:00      12          77.38000

In [61]:
df = df[~df.index.duplicated(keep='first')]


In [62]:
df.drop(columns=['device'], inplace=True)


In [63]:
df = df.resample('15min').mean()

print(f"Total rows after resample: {len(df)}")
print(f"Missing values:\n{df.isnull().sum()}")

Total rows after resample: 5689
Missing values:
externalhumidity    2421
externaltemp        2421
internaltemp        2421
lightintensity      2421
windspeed           2421
winddirection       2421
pressure            2421
altitude            2421
battery             2421
dtype: int64


In [64]:
df.isnull().sum()


externalhumidity    2421
externaltemp        2421
internaltemp        2421
lightintensity      2421
windspeed           2421
winddirection       2421
pressure            2421
altitude            2421
battery             2421
dtype: int64

In [65]:
import numpy as np

def mask_nighttime(df, sleep_hour=21, wake_hour=9):
    hour = df.index.hour
    nighttime = (hour >= sleep_hour) | (hour < wake_hour)
    df.loc[nighttime] = np.nan
    return df

df = mask_nighttime(df)

daytime = df.between_time('09:00', '21:00')
print(f"Daytime rows: {len(daytime)}")
print(f"Daytime missing values:\n{daytime.isnull().sum()}")

Daytime rows: 2916
Daytime missing values:
externalhumidity    834
externaltemp        834
internaltemp        834
lightintensity      834
windspeed           834
winddirection       834
pressure            834
altitude            834
battery             834
dtype: int64


In [66]:
# Find daytime rows where ALL columns are NaN
daytime = df.between_time('09:00', '21:00')
daytime_gaps = daytime[daytime.isnull().all(axis=1)]

print(f"Total missing daytime slots: {len(daytime_gaps)}")
print("\nGap periods:")

# Group consecutive missing slots into gap blocks
gap_index = daytime_gaps.index.to_series()
gap_groups = gap_index.diff().gt(pd.Timedelta('15min')).cumsum()

for group_id, group in daytime_gaps.groupby(gap_groups):
    duration_mins = len(group) * 15
    print(f"  {group.index[0]} → {group.index[-1]}  ({duration_mins} mins / {duration_mins//60}h {duration_mins%60}m)")

Total missing daytime slots: 834

Gap periods:
  2025-12-21 21:00:00+00:00 → 2025-12-21 21:00:00+00:00  (15 mins / 0h 15m)
  2025-12-22 21:00:00+00:00 → 2025-12-22 21:00:00+00:00  (15 mins / 0h 15m)
  2025-12-23 21:00:00+00:00 → 2025-12-23 21:00:00+00:00  (15 mins / 0h 15m)
  2025-12-24 17:45:00+00:00 → 2025-12-24 21:00:00+00:00  (210 mins / 3h 30m)
  2025-12-25 19:00:00+00:00 → 2025-12-25 21:00:00+00:00  (135 mins / 2h 15m)
  2025-12-26 20:00:00+00:00 → 2025-12-26 21:00:00+00:00  (75 mins / 1h 15m)
  2025-12-27 17:30:00+00:00 → 2025-12-27 21:00:00+00:00  (225 mins / 3h 45m)
  2025-12-28 09:00:00+00:00 → 2025-12-28 10:30:00+00:00  (105 mins / 1h 45m)
  2025-12-28 17:30:00+00:00 → 2025-12-28 21:00:00+00:00  (225 mins / 3h 45m)
  2025-12-29 17:45:00+00:00 → 2025-12-29 21:00:00+00:00  (210 mins / 3h 30m)
  2025-12-30 17:00:00+00:00 → 2025-12-30 21:00:00+00:00  (255 mins / 4h 15m)
  2025-12-31 18:30:00+00:00 → 2025-12-31 21:00:00+00:00  (165 mins / 2h 45m)
  2026-01-01 17:00:00+00:00 → 202

In [67]:
def interpolate_short_gaps_only(df, max_gap_slots=4):
    """
    Only interpolate gaps of 4 or fewer consecutive NaN slots (≤1 hour).
    Longer gaps (battery death) remain as NaN.
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].interpolate(
        method='time',
        limit=max_gap_slots,          # won't fill more than 4 consecutive NaNs
        limit_direction='forward',
        limit_area='inside'           # won't fill at edges of the series
    )
    return df

df = interpolate_short_gaps_only(df)

In [68]:
daytime = df.between_time('09:00', '21:00')
print(f"Daytime rows: {len(daytime)}")
print(f"Remaining NaNs after interpolation:\n{daytime.isnull().sum()}")

# What % of daytime is actually usable?
total_daytime = len(daytime)
usable = daytime.dropna(how='all').shape[0]
print(f"\nUsable daytime rows: {usable} / {total_daytime} ({usable/total_daytime*100:.1f}%)")

Daytime rows: 2916
Remaining NaNs after interpolation:
externalhumidity    616
externaltemp        616
internaltemp        616
lightintensity      616
windspeed           616
winddirection       616
pressure            616
altitude            616
battery             616
dtype: int64

Usable daytime rows: 2300 / 2916 (78.9%)


In [69]:
df.to_csv('cleaned_weather_data.csv')

daytime_clean = df.between_time('09:00', '21:00').dropna(how='all')
daytime_clean.to_csv('daytime_weather_data.csv')

print(f"✅ Full dataset saved: {len(df)} rows")
print(f"✅ Daytime dataset saved: {len(daytime_clean)} rows")


✅ Full dataset saved: 5689 rows
✅ Daytime dataset saved: 2300 rows


In [70]:
from prophet import Prophet
import pandas as pd
import numpy as np
import json

In [71]:
df = pd.read_csv('daytime_weather_data.csv', index_col='created_at', parse_dates=True)

df.index = df.index.tz_convert('Asia/Colombo')

parameters = ['externaltemp', 'externalhumidity', 'pressure', 'windspeed', 'lightintensity']

def prepare_prophet_df(df, column):
    """Extract a single column and format for Prophet"""
    prophet_df = df[[column]].dropna().reset_index()
    prophet_df.columns = ['ds', 'y']
    prophet_df['ds'] = prophet_df['ds'].dt.tz_localize(None)
    return prophet_df

In [72]:
def get_model_config(parameter):
    """
    Return Prophet config tuned per parameter.
    - changepoint_prior_scale: how flexible the trend is (higher = more flexible)
    - seasonality_prior_scale: how strongly to fit daily patterns
    """
    configs = {
        'externaltemp': {
            'changepoint_prior_scale': 0.05,
            'seasonality_prior_scale': 10,
            'daily_seasonality': True,
            'weekly_seasonality': True,
            'yearly_seasonality': False,
        },
        'externalhumidity': {
            'changepoint_prior_scale': 0.05,
            'seasonality_prior_scale': 10,
            'daily_seasonality': True,
            'weekly_seasonality': True,
            'yearly_seasonality': False,
        },
        'pressure': {
            'changepoint_prior_scale': 0.01,
            'seasonality_prior_scale': 5,
            'daily_seasonality': True,
            'weekly_seasonality': False,
            'yearly_seasonality': False,
        },
        'windspeed': {
            'changepoint_prior_scale': 0.1,
            'seasonality_prior_scale': 5,
            'daily_seasonality': True,
            'weekly_seasonality': False,
            'yearly_seasonality': False,
        },
        'lightintensity': {
            'changepoint_prior_scale': 0.05,
            'seasonality_prior_scale': 15, 
            'daily_seasonality': True,
            'weekly_seasonality': False,
            'yearly_seasonality': False,
        },
    }
    return configs[parameter]


def train_model(df, parameter):
    prophet_df = prepare_prophet_df(df, parameter)
    config = get_model_config(parameter)

    model = Prophet(**config)
    model.fit(prophet_df)

    print(f"✅ {parameter} model trained on {len(prophet_df)} rows")
    return model

In [73]:
def forecast_24h(model, parameter):

    last_timestamp = pd.Timestamp.now().normalize()
    future_times = pd.date_range(
        start=last_timestamp,
        periods=96, 
        freq='15min'
    )

    future_df = pd.DataFrame({'ds': future_times})
    forecast = model.predict(future_df)

    result = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
    result.columns = ['timestamp', 'predicted', 'lower_bound', 'upper_bound']

    result['is_daytime'] = result['timestamp'].dt.hour.between(9, 20)
    result.loc[~result['is_daytime'], ['predicted', 'lower_bound', 'upper_bound']] = None

    result[['predicted', 'lower_bound', 'upper_bound']] = result[
        ['predicted', 'lower_bound', 'upper_bound']
    ].round(2)

    result['parameter'] = parameter
    return result

In [74]:
forecasts = {}
models = {}

for param in parameters:
    print(f"\nTraining {param}...")
    model = train_model(df, param)
    models[param] = model
    forecasts[param] = forecast_24h(model, param)

print("\n✅ All models trained and forecasts generated")


Training externaltemp...


12:30:58 - cmdstanpy - INFO - Chain [1] start processing
12:31:00 - cmdstanpy - INFO - Chain [1] done processing


✅ externaltemp model trained on 2300 rows

Training externalhumidity...


12:31:01 - cmdstanpy - INFO - Chain [1] start processing
12:31:01 - cmdstanpy - INFO - Chain [1] done processing
12:31:02 - cmdstanpy - INFO - Chain [1] start processing


✅ externalhumidity model trained on 2300 rows

Training pressure...


...

✅ pressure model trained on 2300 rows

Training windspeed...


12:31:03 - cmdstanpy - INFO - Chain [1] done processing
12:31:03 - cmdstanpy - INFO - Chain [1] start processing


✅ windspeed model trained on 2300 rows

Training lightintensity...


12:31:03 - cmdstanpy - INFO - Chain [1] done processing


✅ lightintensity model trained on 2300 rows

✅ All models trained and forecasts generated


In [75]:
forecasts

{'externaltemp':              timestamp  predicted  lower_bound  upper_bound  is_daytime  \
 0  2026-02-19 00:00:00        NaN          NaN          NaN       False   
 1  2026-02-19 00:15:00        NaN          NaN          NaN       False   
 2  2026-02-19 00:30:00        NaN          NaN          NaN       False   
 3  2026-02-19 00:45:00        NaN          NaN          NaN       False   
 4  2026-02-19 01:00:00        NaN          NaN          NaN       False   
 ..                 ...        ...          ...          ...         ...   
 91 2026-02-19 22:45:00        NaN          NaN          NaN       False   
 92 2026-02-19 23:00:00        NaN          NaN          NaN       False   
 93 2026-02-19 23:15:00        NaN          NaN          NaN       False   
 94 2026-02-19 23:30:00        NaN          NaN          NaN       False   
 95 2026-02-19 23:45:00        NaN          NaN          NaN       False   
 
        parameter  
 0   externaltemp  
 1   externaltemp  
 2   exter